# Visualização 2D com Autoencoder
Neste notebook, vamos:
1. Gerar 4 distribuições gaussianas de 8 dimensões com médias distintas.
2. Treinar uma rede **autoencoder** para reduzir essas 8 dimensões para 2.
3. Visualizar os dados codificados em 2D.

### 📌 Introdução
Este exercício envolve o uso de redes neurais autoencoders para realizar redução de dimensionalidade de dados. Vamos trabalhar com quatro distribuições gaussianas em um espaço de 8 dimensões, com médias distintas, e utilizaremos um autoencoder para reduzir essa dimensionalidade para duas dimensões, facilitando a visualização dos dados. A seguir, temos as etapas e respostas para cada item do exercício.

### a) Definição das Distribuições Gaussianas
O exercício começa com quatro distribuições gaussianas, cada uma com média diferente em um espaço de 8 dimensões. As médias dessas distribuições são:

1. **Média de $C_1$:** $\mathbf{m}_1 = (0, 0, 0, 0, 0, 0, 0, 0)$
2. **Média de $C_2$:** $\mathbf{m}_2 = (4, 0, 0, 0, 0, 0, 0, 0)$
3. **Média de $C_3$:** $\mathbf{m}_3 = (0, 0, 4, 0, 0, 0, 0, 0)$
4. **Média de $C_4$:** $\mathbf{m}_4 = (0, 0, 0, 0, 0, 0, 0, 4)$

Cada distribuição tem variância unitária e suas amostras são geradas com base nessas médias.

### b) Utilizando o Autoencoder para Visualização
Para reduzir a dimensionalidade dos dados de 8 para 2 dimensões, utilizamos uma rede neural autoencoder. O autoencoder foi treinado para codificar os dados em um espaço de 2 dimensões e reconstrui-los a partir dessa codificação. O objetivo é preservar a estrutura e as relações dos dados enquanto os projetamos para uma dimensão mais baixa.

A arquitetura do autoencoder é composta por duas partes:
- **Encoder:** que reduz a dimensionalidade dos dados de 8 para 2.
- **Decoder:** que reconstrói os dados de volta para 8 dimensões.


### c) Objetivo: Visualização em 2D
O objetivo principal é transformar dados originalmente em 8 dimensões em um novo espaço bidimensional, de forma que a visualização dos dados em 2D facilite a análise. A redução de dimensionalidade preserva a estrutura dos dados para que as classes possam ser separadas de forma clara.


### d) Apresentação dos Dados no Novo Espaço
Após o treinamento do autoencoder, projetamos os dados em 2 dimensões e os visualizamos utilizando um gráfico de dispersão. O gráfico resultante mostra as quatro classes de distribuições gaussianas, e as cores representam as diferentes classes $C_1$, $C_2$, $C_3$, $C_4$. As classes são visualmente separáveis, o que confirma que o autoencoder conseguiu reduzir com sucesso a dimensionalidade preservando as relações entre as distribuições.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Reprodutibilidade
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# Médias das distribuições gaussianas
means = [
    np.array([0, 0, 0, 0, 0, 0, 0, 0]),
    np.array([4, 0, 0, 0, 0, 0, 0, 0]),
    np.array([0, 0, 4, 0, 0, 0, 0, 0]),
    np.array([0, 0, 0, 0, 0, 0, 0, 4])
]

num_samples_per_class = 500
X, y = [], []

# Gerar amostras com variância unitária
for i, mean in enumerate(means):
    cov = np.eye(8)
    samples = np.random.multivariate_normal(mean, cov, num_samples_per_class)
    X.append(samples)
    y.append(np.full(num_samples_per_class, i))

X = np.vstack(X)
y = np.concatenate(y)
X_tensor = torch.tensor(X, dtype=torch.float32)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 8)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

# Instanciar modelo
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, X_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
with torch.no_grad():
    encoded = model.encoder(X_tensor).numpy()

In [ ]:
plt.figure(figsize=(8, 6))
colors = ['red', 'green', 'blue', 'orange']
labels = ['C1', 'C2', 'C3', 'C4']

for i in range(4):
    plt.scatter(encoded[y == i, 0], encoded[y == i, 1],
                label=labels[i], alpha=0.6, color=colors[i])

plt.title('Visualização 2D com Autoencoder')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()